pip install transformers

In [2]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def tokenize_text(text):
  marked_text = "[CLS] " + text + " [SEP]"
  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [1] * len(tokenized_text)

  #print(len(tokenized_text), " in total")

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  return tokenized_text, tokens_tensor, segments_tensors

In [11]:
def paragraph_to_vec(text):
  tokenized_text, tokens_tensor, segments_tensors = tokenize_text(text)

  outputs = model(tokens_tensor, segments_tensors)

  hidden_states = outputs[2]
  token_vecs = hidden_states[-2][0]
  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding

In [13]:
text= 'With probability 0.5, sentence A and sentence B correspond to two consecutive sentences in the original corpus'
text_vec = paragraph_to_vec(text)
len(text_vec), text_vec

(768, tensor([-3.8795e-01, -1.5764e-01,  2.9915e-01, -5.7462e-01, -1.0337e-01,
          1.3365e-01,  3.7756e-01,  5.0848e-02, -1.0800e-02, -1.9605e-01,
          7.2315e-02, -5.7666e-02, -3.4052e-01,  3.3552e-01,  6.5739e-02,
          4.0309e-01,  3.7748e-01,  8.6403e-02, -8.1855e-02, -3.2563e-01,
          6.8994e-01,  3.9178e-01, -8.4165e-01, -2.4641e-02,  5.0810e-01,
         -2.4833e-01,  9.0553e-02,  1.9387e-01, -1.9859e-01, -2.2789e-02,
         -2.7723e-01,  4.2261e-01,  4.1581e-01, -2.2254e-01,  2.2010e-01,
          1.6702e-01,  2.5255e-01, -4.6958e-01, -7.8662e-03,  4.3897e-01,
         -2.1923e-01,  1.3300e-01, -2.0808e-01, -6.3029e-01, -4.3113e-01,
         -7.4590e-01,  3.7591e-02, -9.6667e-02,  3.1609e-02, -4.5862e-01,
         -8.9934e-01, -9.3408e-02, -4.2174e-01,  4.3909e-01, -2.4055e-01,
          7.3370e-01,  6.5822e-02, -6.2039e-01,  3.7215e-01,  7.1766e-02,
         -2.7811e-01, -1.5417e-01,  3.4252e-01,  7.7989e-02,  1.7137e-01,
         -2.5580e-02,  3.2224e-01